In [19]:
import pandas as pd
import numpy as np
import re

In [20]:
def tables_in_query(sql_str):

    # remove the /* */ comments
    q = re.sub(r"/\*[^*]*\*+(?:[^*/][^*]*\*+)*/", "", sql_str)

    # remove whole line -- and # comments
    lines = [line for line in q.splitlines() if not re.match("^\s*(--|#)", line)]

    # remove trailing -- and # comments
    q = " ".join([re.split("--|#", line)[0] for line in lines])

    # add space between commas if not space existis 
    q = re.sub(r'[,]', r', ', q)
    
    # split on blanks, parens and semicolons
    tokens = re.split(r"[\s)(;]+", q)

    # scan the tokens. if we see a FROM or JOIN, we set the get_next
    # flag, and grab the next one (unless it's SELECT).

    result = set()
    get_next = False
    from_ident = False
    for tok in tokens:
        comma = ',' in tok
        if (tok.lower() in ['from']):
            from_ident = True
        elif (tok.lower() in ['select','order','where','group','having','limit', 'in']):
            from_ident= False
#         print(tok,'-', get_next,'-', comma, '-', from_ident)
        if (get_next & from_ident):
            if tok.lower() not in ["", "select"]:
#                 print('tok to be added', tok)
                tok =re.sub(r'[,]', r'', tok)
#                 print('tok to be added', tok)
                result.add(tok)
        if comma:
            get_next = True
        else:
            get_next = tok.lower() in ["from", "join"]
#     print(result)
    return result

In [21]:
column = ['user', 'sql_query']

In [22]:
list_of_lists = [
    ['user1', 'CREATE TABLE EDWARD_FIX_WGS_STAR AS SELECT group_id ,MBRSHP_SOR_CD ,legal_entity as LEGAL_ENTITY2 ,line_of_business AS LINE_OF_BUSINESS2 ,mbu AS MBU2 ,funding as funding2 ,MAX(service_date) AS max_date FROM final2 WHERE LEGAL_ENTITY NOT IN (‘UNK’,‘’) AND MBU NOT IN (‘UNK’,‘’) AND substr(MBU,1,2) NOT IN (‘LG’) AND mbrshp_sor_cd IN (‘808’,‘815’,‘877’)   /*  WGS STAR  QCARE_NOTREADY  */ GROUP BY 1,2,3,4,5 ORDER BY 1,2,7 DESC'],
    ['user2', 'SELECT * FROM table.table1 LEFT JOIN table2;'],
    ['user3', 'SELECT ID FROM Customers INNER JOIN Orders'],
    ['','select K.a,K.b from (select H.b from H), I, J, K join L on order by 1, 2;'],
]

In [23]:
sql_query_dataframe = pd.DataFrame(list_of_lists, columns=column)

In [24]:
sql_query_dataframe = pd.read_csv('sql_queries.csv')

In [25]:
sql_query_dataframe['tables']= ''

In [26]:
sql_query_dataframe.head()

,sql_query,tables
0,CREATE TABLE diag_cd_dom AS SELECT * FROM CONN...,
1,CREATE TABLE diag_cd_dom AS SELECT * FROM CONN...,
2,CREATE TABLE deny_dom AS SELECT * FROM CONNECT...,
3,CREATE TABLE CPT_dom AS SELECT * FROM CONNECTI...,
4,CREATE TABLE Drug_name AS SELECT * FROM CONNEC...,


In [27]:
sql_query_dataframe.shape

(110, 2)

In [28]:
# sql_query_dataframe

In [29]:
for index, row in sql_query_dataframe.iterrows():
    row['tables']= tables_in_query(row.sql_query)

In [30]:
sql_query_dataframe

,sql_query,tables
0,CREATE TABLE diag_cd_dom AS SELECT * FROM CONN...,"{EDW_ALLPHI.DIAG, CONNECTION}"
1,CREATE TABLE diag_cd_dom AS SELECT * FROM CONN...,"{EDW_ALLPHI.DIAG, CONNECTION}"
2,CREATE TABLE deny_dom AS SELECT * FROM CONNECT...,"{CONNECTION, EDW_ALLPHI.SRC_SRVC_DNL_RSN_CD}"
3,CREATE TABLE CPT_dom AS SELECT * FROM CONNECTI...,"{edw_allphi.hlth_srvc, CONNECTION}"
4,CREATE TABLE Drug_name AS SELECT * FROM CONNEC...,"{EDW_ALLPHI.NDC, CONNECTION}"
5,CREATE TABLE Pharmacy_name AS SELECT * FROM CO...,"{CONNECTION, EDW_ALLPHI.PROV_ADRS}"
6,CREATE TABLE Mike.Qcare_PROV_name AS SELECT * ...,{CONNECTION}
7,CREATE TABLE EDWARDa AS \nSELECT * FROM CONNEC...,"{EDW_ALLPHI.CLM_PROV, '~NA', EDW_ALLPHI.CLM_PA..."
8,CREATE TABLE EDWARDb AS \nSELECT * FROM CONNEC...,"{'~NA', EDW2_ALLPHI.CLM_line_PROV, EDW_ALLPHI...."
9,"DISP_CD""",{}


In [31]:
sql_query_dataframe.to_csv('results.csv')

In [34]:
# tables_in_query('SELECT * from Evalulation e JOIN Value v ON e.CaseNum = v.CaseNum \
#     AND e.FileNum = v.FileNum AND e.ActivityNum = v.ActivityNum')

In [624]:
# tables_in_query('CREATE TABLE diag_cd_dom AS SELECT * FROM CONNECTION TO TERADATA \
#   (select Z.diag_cd                as prim_diag_cd1, \
#           Z.diag_cd                as diag_2, \
#  Z.diag_cd                as diag_3, \
#   Z.diag_cd                as diag_4, \
#   Z.diag_cd                as diag_5, \
#   Z.diag_cd                as diag_6, \
#                 Z.diag_cd                as diag_7, \
#   Z.diag_cd                as diag_8, \
#   Z.diag_cd                as diag_9, \
#           z.ICD_VRSN_CD            as ICD_VRSN_CD, \
#  UPPER(Z.icd_full_desc)  as diag_cd_desc \
#    from  EDW_ALLPHI.DIAG Z \
# /*   where z.ICD_VRSN_CD=9*/ \
#   group by 1,2,3,4,5,6,7,8,9,10,11 \
#   order by 1,10);')

In [561]:
# tables_in_query('select K.a,K.b from (select H.b from H), I,J,K join L on order by 1, 2;')

In [562]:
# tables_in_query('SELECT pid, cid, pname, name1, name2 FROM customer1 c1, product p')

In [563]:
# tables_in_query('CREATE TABLE EDWARD_FIX_WGS_STAR AS SELECT group_id ,MBRSHP_SOR_CD ,legal_entity as LEGAL_ENTITY2 ,line_of_business AS LINE_OF_BUSINESS2 ,mbu AS MBU2 ,funding as funding2 ,MAX(service_date) AS max_date FROM final2 WHERE LEGAL_ENTITY NOT IN (‘UNK’,‘’) AND MBU NOT IN (‘UNK’,‘’) AND substr(MBU,1,2) NOT IN (‘LG’) AND mbrshp_sor_cd IN (‘808’,‘815’,‘877’)   /*  WGS STAR  QCARE_NOTREADY  */ GROUP BY 1,2,3,4,5 ORDER BY 1,2,7 DESC;')